# Prostate cancer dataset

**References:**
- Hastie, Tibshiranie, Friedman: The Elements of Statistical Learning, Section 3.2.1

**Data source:** Stamey et al (1989)

https://web.stanford.edu/~hastie/ElemStatLearn/data.html

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing
%matplotlib inline

## Load in dataset

In [2]:
data_path = './prostate.data'

In [3]:
data_all = pd.read_csv(data_path, delim_whitespace=True)

In [4]:
len(data_all)

97

In [5]:
with open('./prostate.info.txt', 'r') as f:
    data_description = f.read()

In [6]:
print(data_description)

Prostate data info

Predictors (columns 1--8)

lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45

outcome (column 9)

lpsa

train/test indicator (column 10)

This last column indicates which 67 observations were used as the 
"training set" and which 30 as the test set, as described on page 48
in the book.

There was an error in these data in the first edition of this
book. Subject 32 had a value of 6.1 for lweight, which translates to a
449 gm prostate! The correct value is 44.9 gm. We are grateful to
Prof. Stephen W. Link for alerting us to this error.

The features must first be scaled to have mean zero and  variance 96 (=n)
before the analyses in Tables 3.1 and beyond.  That is, if x is the  96 by 8 matrix
of features, we compute xp <- scale(x,TRUE,TRUE)




In [7]:
np.log(44.9)

3.8044377947482086

In [8]:
data_all.iloc[31]

lcavol     0.182322
lweight     3.80444
age              65
lbph        1.70475
svi               0
lcp        -1.38629
gleason           6
pgg45             0
lpsa        2.00821
train             F
Name: 32, dtype: object

In [9]:
data_all.head(2)

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T


------------

## Preprocessing

### Creating transformer to standardize predictors

In [10]:
scaler = preprocessing.StandardScaler().fit(data_all.drop(columns=['train', 'lpsa']))

In [11]:
scaler.mean_

array([ 1.35000958,  3.62894266, 63.86597938,  0.10035561,  0.21649485,
       -0.17936558,  6.75257732, 24.3814433 ])

In [12]:
scaler.scale_

array([ 1.17253375,  0.4261972 ,  7.40664075,  1.44330887,  0.41185535,
        1.39102348,  0.71840212, 28.05827636])

In [13]:
#scaler.transform(data_all.drop(columns=['train'])  )[:,0].std()

-------------

### Splitting the full dataset to training and test datasets

In [14]:
data_train = data_all[data_all['train'] == 'T'].drop(columns=['train'])
data_test = data_all[data_all['train'] == 'F'].drop(columns=['train'])

In [15]:
len(data_train), len(data_test)

(67, 30)

In [16]:
#data_train_transformed.mean(axis=1)

In [17]:
#scaler.

In [18]:
y_train = data_train['lpsa']
X_train = data_train.drop(columns=['lpsa'])

In [19]:
y_test = data_test['lpsa']
X_test = data_test.drop(columns=['lpsa'])

### Standardization of predictiors

In [20]:
X_test_transformed = scaler.transform(X_test)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=X_test.columns, index=X_test.index)

In [21]:
#X_train_transformed.tail(2)


### Add constant array

In [22]:
endog_train = y_train
engog_test = y_test
exog_train = sm.add_constant(X_train_transformed)
exog_test = sm.add_constant(X_test_transformed)

NameError: name 'X_train_transformed' is not defined

------

## Fitting the OLS model

In [ ]:
model = sm.OLS(endog=endog_train, exog=exog_train)

In [ ]:
res = model.fit()

In [ ]:
res.mse_resid

In [ ]:
res.mse_total

In [ ]:
res.mse_model

In [ ]:
#res.

In [ ]:
print(res.summary())

-----------------------------

## F-test on the non-significant variables

Non-significant variables:
- `age`
- `lcp`
- `gleason`
- `pgg45`

**Signature:**
`res.f_test(r_matrix, cov_p=None, scale=1.0, invcov=None)`

**Docstring:**
Compute the F-test for a joint linear hypothesis.

This is a special case of `wald_test` that always uses the F
distribution.

Parameters
----------
- `r_matrix` : array-like, str, or tuple
    - array : An r x k array where r is the number of restrictions to
      test and k is the number of regressors. It is assumed
      that the linear combination is equal to zero.
    - str : The full hypotheses to test can be given as a string.
      See the examples.
    - tuple : A tuple of arrays in the form (R, q), ``q`` can be
      either a scalar or a length k row vector.
- `cov_p` : array-like, optional
    An alternative estimate for the parameter covariance matrix.
    If None is given, self.normalized_cov_params is used.
- `scale` : float, optional

    .. deprecated:: 0.10.0

    Default is 1.0 for no scaling.

- `invcov` : array-like, optional
    A q x q array to specify an inverse covariance matrix based on a
    restrictions matrix.


Returns
-------
- `res`: `ContrastResults` instance
    The results for the test are attributes of this results instance.

In [ ]:
exog_train.head(2)

In [ ]:
exog_train.columns[ [3, 6, 7, 8 ] ]

In [ ]:
r_matrix = [[0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1]]

In [ ]:
res.f_test(r_matrix)

In [ ]:
r_matrix = "age = lcp = gleason = pgg45 = 0"

In [ ]:
res.f_test(r_matrix)

Values in ESL:

F = 1.67

\begin{equation}
    \textrm{Prob} \left( F_{4,58} > 1.67 \right) = 0.17
\end{equation}

----------------------------

## Error rates

### Linear regression model error rate

In [ ]:
y_test_pred = res.predict(exog=exog_test)

In [ ]:
RSS_linreg = np.sum( (y_test_pred - y_test)**2.0)/len(y_test)
RSS_linreg

### Base error rate

In [ ]:
y_train_mean = y_train.mean()

In [ ]:
RSS_base = np.sum( (y_train_mean - y_test)**2.0)/len(y_test)
RSS_base

Decrease in RSS, the base model vs. linear regression

In [ ]:
(RSS_linreg/RSS_base - 1.0)*100.0